
# Requirements
Shows birds characteristics (sounds or pictures) and quizes you on the name of the bird.
This will be pulling from a set database (not from the API)
# Data needed
- bird songs and calls audiofiles
- pictures of each bird
- names of each bird


# Set list of birds

In [31]:
bird_names = ["Haemorhous mexicanus", "Sayornis nigricans", "Tyrannus vociferans", "Piranga ludoviciana", "Pheucticus melanocephalus", "Spinus psaltria", "Spinus tristis", "Passer domesticus", "Melospiza melodia", "Melozone crissalis", "Setophaga petechia", "Vireo bellii", "Polioptila caerulea", "Troglodytes aedon", "Thryomanes bewickii", "Baeolophus inornatus", "Psaltriparus minimus", "Chamaea fasciata", "Pycnonotus jocosus", "Phainopepla nitens", "Sialia mexicana", "Turdus migratorius", "Aphelocoma californica", "Dryobates nuttallii", "Hirundo rustica", "Calypte anna", "Selasphorus sasin", "Buteo lineatus"] 


# Download songs

In [42]:
import requests


bird_name = "Pycnonotus jocosus"
url = f"https://xeno-canto.org/api/2/recordings?query={bird_name.replace(' ', '+')}+q:\">C\""

response = requests.get(url)

print(response)


<Response [200]>


In [43]:
response_body = response.json()

In [44]:
response_body

{'numRecordings': '188',
 'numSpecies': '1',
 'page': 1,
 'numPages': 1,
 'recordings': [{'id': '882108',
   'gen': 'Pycnonotus',
   'sp': 'jocosus',
   'ssp': 'jocosus',
   'group': 'birds',
   'en': 'Red-whiskered Bulbul',
   'rec': 'Geoff Carey',
   'cnt': 'China',
   'loc': 'Pak Sha O, Sai Kung District, New Territories, Hong Kong',
   'lat': '22.4495',
   'lng': '114.3209',
   'alt': '70',
   'type': 'call',
   'sex': '',
   'stage': '',
   'method': 'field recording',
   'url': '//xeno-canto.org/882108',
   'file': 'https://xeno-canto.org/882108/download',
   'file-name': 'XC882108-Pycnonotus-jocosus-jocosus-T4878-norm-8dB-NR-12dB-HP-500Hz.mp3',
   'sono': {'small': '//xeno-canto.org/sounds/uploaded/LBWCUENEBH/ffts/XC882108-small.png',
    'med': '//xeno-canto.org/sounds/uploaded/LBWCUENEBH/ffts/XC882108-med.png',
    'large': '//xeno-canto.org/sounds/uploaded/LBWCUENEBH/ffts/XC882108-large.png',
    'full': '//xeno-canto.org/sounds/uploaded/LBWCUENEBH/ffts/XC882108-full.png'},
 

In [45]:
# make a function that takes a scientific name of a bird and automatically downloads all audio files for that bird  
# and adds entries to the bird data CSV

#Step 1: Generate URL with bird of interest
#step 2: navigate to audio file in response body JSON
#step 3: loop over inner bits of information within the 'recording' list

import requests
from tqdm import tqdm
import pandas as pd
import os

def identify_bird_song(sci_bird_name):
    calls_songs = []
    url = f"https://xeno-canto.org/api/2/recordings?query={sci_bird_name.replace(' ', '+')}+q:\">C\""
    response = requests.get(url)
    response_body = response.json()
    
    if os.path.exists('data/bird_data.csv'):
      known_files = pd.read_csv('data/bird_data.csv')['file_path'].tolist()
    else:
      known_files = []
    for recording in tqdm(response_body['recordings']):
        
        if recording['cnt'] == "United States" and int(recording['length'].split(':')[0]) <= 0 and recording['date'] >= '2015' and recording['type'] in ['song', 'dawn song', 'call'] and f"{recording['id']}.mp3" not in known_files:
          rel_data = {}
          rel_data['audio_url'] = recording['file']
          rel_data['vocalization_type'] = recording['type']
          rel_data['length'] = recording['length']
          rel_data['date'] = recording['date']
          rel_data['file_path'] = f"{recording['id']}.mp3"
          rel_data['sci_name'] = sci_bird_name
          rel_data['common_name'] = recording['en']
            
          calls_songs.append(rel_data)

          bird_sound_url = recording['file']
          bird_sound_mp3 = requests.get(bird_sound_url)
          
          with open(f"data/{recording['id']}.mp3", 'wb') as fd:
            for chunk in bird_sound_mp3.iter_content(chunk_size=128):
              fd.write(chunk)
          
    # Load dataframe (what the edge cases--rare situations to account for--here)
    if os.path.exists('data/bird_data.csv'):
      bird_data = pd.read_csv('data/bird_data.csv')
      new_bird_data = pd.DataFrame(calls_songs)

      # Combine with new data
      bird_data = pd.concat([bird_data, new_bird_data])

      # Dedup data
      bird_data = bird_data.drop_duplicates()
    else:
      bird_data = pd.DataFrame(calls_songs)




    # Save data
    bird_data.to_csv('data/bird_data.csv', index=False)
    
identify_bird_song("Piranga ludoviciana")   




['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

100%|██████████| 182/182 [01:15<00:00,  2.40it/s]


In [46]:
# For each observation in the response_body, determine if it is in california

for bird in bird_names:
    identify_bird_song(bird)




['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

100%|██████████| 410/410 [02:03<00:00,  3.31it/s]


['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

100%|██████████| 165/165 [01:19<00:00,  2.07it/s]


['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

...

['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

100%|██████████| 182/182 [00:00<00:00, 258504.34it/s]


['878437.mp3', '807072.mp3', '776027.mp3', '651934.mp3', '651609.mp3', '582726.mp3', '558376.mp3', '537679.mp3', '537678.mp3', '494385.mp3', '494384.mp3', '441368.mp3', '424053.mp3', '424052.mp3', '408766.mp3', '355404.mp3', '319003.mp3', '928000.mp3', '899902.mp3', '898674.mp3', '875451.mp3', '814493.mp3', '658900.mp3', '622745.mp3', '584861.mp3', '584860.mp3', '572692.mp3', '570952.mp3', '500927.mp3', '498247.mp3', '493111.mp3', '474138.mp3', '471147.mp3', '468779.mp3', '423324.mp3', '362519.mp3', '358850.mp3', '319140.mp3', '310666.mp3', '309624.mp3', '301685.mp3', '898666.mp3', '875440.mp3', '842469.mp3', '624591.mp3', '528242.mp3', '452150.mp3', '451252.mp3', '450999.mp3', '398477.mp3', '384905.mp3', '357024.mp3', '353668.mp3', '351287.mp3', '297441.mp3', '907177.mp3', '899884.mp3', '805787.mp3', '757512.mp3', '649062.mp3', '646101.mp3', '576567.mp3', '574235.mp3', '387756.mp3', '358681.mp3', '351546.mp3', '321775.mp3', '911995.mp3', '906426.mp3', '901447.mp3', '807036.mp3', '8028

 90%|████████▉ | 311/347 [20:58:38<2:25:41, 242.83s/it]


KeyboardInterrupt: 

In [59]:
calls_songs

[{'audio_url': 'https://xeno-canto.org/900144/download',
  'vocalization_type': 'call, song',
  'length': '0:28',
  'date': '2024-04-24',
  'sex': 'uncertain',
  'file_path': '900144.mp3'},
 {'audio_url': 'https://xeno-canto.org/900139/download',
  'vocalization_type': 'call, chirp',
  'length': '0:25',
  'date': '2024-04-24',
  'sex': 'uncertain',
  'file_path': '900139.mp3'},
 {'audio_url': 'https://xeno-canto.org/899902/download',
  'vocalization_type': 'song',
  'length': '0:52',
  'date': '2024-05-02',
  'sex': 'male',
  'file_path': '899902.mp3'},
 {'audio_url': 'https://xeno-canto.org/898674/download',
  'vocalization_type': 'song',
  'length': '0:40',
  'date': '2024-04-25',
  'sex': 'male',
  'file_path': '898674.mp3'},
 {'audio_url': 'https://xeno-canto.org/875451/download',
  'vocalization_type': 'song',
  'length': '0:56',
  'date': '2024-02-16',
  'sex': 'male',
  'file_path': '875451.mp3'},
 {'audio_url': 'https://xeno-canto.org/743188/download',
  'vocalization_type': 'c

In [62]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 42.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 23.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 12.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [72]:
import pandas as pd

pd.DataFrame(calls_songs).to_csv('data/bird_data.csv')

In [51]:
int('0:59'.split(':')[0]) <= 0

True

In [28]:
response_body

{'numRecordings': '478',
 'numSpecies': '1',
 'page': 1,
 'numPages': 1,
 'recordings': [{'id': '531575',
   'gen': 'Haemorhous',
   'sp': 'mexicanus',
   'ssp': '',
   'group': 'birds',
   'en': 'House Finch',
   'rec': 'Thomas Magarian',
   'cnt': 'United States',
   'loc': 'Madrona City Park, Portland, Multnomah County, Oregon',
   'lat': '45.5582',
   'lng': '-122.6928',
   'alt': '60',
   'type': 'call',
   'sex': '',
   'stage': '',
   'method': 'field recording',
   'url': '//xeno-canto.org/531575',
   'file': 'https://xeno-canto.org/531575/download',
   'file-name': 'XC531575-HOFI_2019-03-07_Willamette_Bluff_North_Portland_OR_1023.mp3',
   'sono': {'small': '//xeno-canto.org/sounds/uploaded/GVVNEJJEGA/ffts/XC531575-small.png',
    'med': '//xeno-canto.org/sounds/uploaded/GVVNEJJEGA/ffts/XC531575-med.png',
    'large': '//xeno-canto.org/sounds/uploaded/GVVNEJJEGA/ffts/XC531575-large.png',
    'full': '//xeno-canto.org/sounds/uploaded/GVVNEJJEGA/ffts/XC531575-full.png'},
   'osci

In [20]:
pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 381.9 kB/s eta 0:00:00:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from tqdm import tqdm
import requests

url = "https://xeno-canto.org/531575/download"
response = requests.get(url, stream=True)

with open("test_file.mp3", "wb") as handle:
    for data in tqdm(response.iter_content()):
        handle.write(data)

118722it [00:00, 129553.75it/s]


# Download calls

In [25]:
bird_data = pd.read_csv('data/bird_data.csv')

print(bird_data)

                                 audio_url vocalization_type length  \
0   https://xeno-canto.org/807072/download              song   0:33   
1   https://xeno-canto.org/408766/download              call   0:35   
2   https://xeno-canto.org/878437/download              song   0:45   
3   https://xeno-canto.org/807072/download              song   0:33   
4   https://xeno-canto.org/776027/download              song   0:27   
5   https://xeno-canto.org/651934/download              call   0:28   
6   https://xeno-canto.org/651609/download              song   0:48   
7   https://xeno-canto.org/582726/download              call   0:42   
8   https://xeno-canto.org/558376/download              song   0:37   
9   https://xeno-canto.org/537679/download              song   0:44   
10  https://xeno-canto.org/537678/download              song   0:30   
11  https://xeno-canto.org/494385/download              song   0:45   
12  https://xeno-canto.org/494384/download              song   0:13   
13  ht

# Download Pictures